# Determine Top Indictators of Heart Disease Using Multiple Linear Regressors
Author: Tommy Phung
Soruce: Kaggle*

Overview

# Data Exploration
The data was provided by Bhavesh Misra on Kaggle with multiple different columns. The idea of this dataset was to see if we can correlate the different features and attempt to connect them having a heart disease or heart attack. Its important to note that some of the columns are a little ambigious and that it will not be used for the purpose not misintretaing the data or author intentions. 

| **Column Name** | **Description** | **Markers** |
|:---:|:---:|:---:|
| HeartDiseaseorAttack: | Indicates whether the individual has had a heart disease or heart attack | (binary: 0 = No, 1 = Yes) |
| HighBP | High blood pressure status | (binary: 0 = No, 1 = Yes) |
| HighChol | High cholesterol status | (binary: 0 = No, 1 = Yes) |
| CholCheck | Frequency of cholesterol check | categorical |
| BMI | Body Mass Index | continuous |
| Smoker | Smoking status | (binary: 0 = No, 1 = Yes) |
| Stroke | History of stroke | (binary: 0 = No, 1 = Yes) |
| Diabetes | Diabetes status | (binary: 0 = No, 1 = Yes) |
| PhysActivity | Level of physical activity | categorical |
| Fruits | Frequency of fruit consumption | categorical |
| Veggies | Frequency of vegetable consumption | categorical |
| HvyAlcoholConsump | Heavy alcohol consumption status | (binary: 0 = No, 1 = Yes) |
| AnyHealthcare | Access to any healthcare | (binary: 0 = No, 1 = Yes) |
| NoDocbcCost | No doctor because of cost | (binary: 0 = No, 1 = Yes) |
| GenHlth | General health assessment | categorical |
| MentHlth | Mental health assessment | categorical |
| PhysHlth | Level of physical activity | categorical |
| DiffWalk | Difficulty walking status | binary: 0 = No, 1 = Yes |
| Sex | Gender of the individual | binary: 0 = Female, 1 = Male |
| Age | Age of the individual | continuous |
| Education | Educational level | categorical |
| Income | Income level | categorical |

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

In [3]:
df = pd.read_csv('data/heart_disease_health_indicators.csv')
display(df.head())
display(df.info())


,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0,1,1,1,40,1,0,0,0,0,...,1,0,5,18,15,1,0,9,4,3
1,0,0,0,0,25,1,0,0,1,0,...,0,1,3,0,0,0,0,7,6,1
2,0,1,1,1,28,0,0,0,0,1,...,1,1,5,30,30,1,0,9,4,8
3,0,1,0,1,27,0,0,0,1,1,...,1,0,2,0,0,0,0,11,3,6
4,0,1,1,1,24,0,0,0,1,1,...,1,0,2,3,0,0,0,11,5,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253661 entries, 0 to 253660
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype
---  ------                --------------   -----
 0   HeartDiseaseorAttack  253661 non-null  int64
 1   HighBP                253661 non-null  int64
 2   HighChol              253661 non-null  int64
 3   CholCheck             253661 non-null  int64
 4   BMI                   253661 non-null  int64
 5   Smoker                253661 non-null  int64
 6   Stroke                253661 non-null  int64
 7   Diabetes              253661 non-null  int64
 8   PhysActivity          253661 non-null  int64
 9   Fruits                253661 non-null  int64
 10  Veggies               253661 non-null  int64
 11  HvyAlcoholConsump     253661 non-null  int64
 12  AnyHealthcare         253661 non-null  int64
 13  NoDocbcCost           253661 non-null  int64
 14  GenHlth               253661 non-null  int64
 15  MentHlth              253661 non-n

None

# Data Preperation
The data comprise of continous, catergorial and binary based columns. In order to perform linear regression on this sort of data, the categorical and binary data must be change to one hot encoding before modeling. There wasn't any missing data from any of the columns so there isn't any data that needed to be removed or changed. 

In [5]:
cont_col = ['BMI', 'Age']
cat_col = ['Education', 'Income', 'Diabetes']
bin_col = ['CholCheck', 'PhysActivity', 'Fruits', 'Veggies', 'HighBP', 'HighChol', 'Smoker', 
           'Stroke', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex']
on_hold = ['GenHlth','MentHlth', 'PhysHlth']
target = 'HeartDiseaseorAttack'

In [6]:
cat_df = df[cat_col]
bin_df = df[bin_col]
cont_df = df[cont_col]

# Baseline Model

In [10]:
X = df[cont_col]
y = df[target]
model = sm.OLS(y, sm.add_constant(X))
result = model.fit()

In [12]:
print(result.summary())

                             OLS Regression Results                             
Dep. Variable:     HeartDiseaseorAttack   R-squared:                       0.053
Model:                              OLS   Adj. R-squared:                  0.053
Method:                   Least Squares   F-statistic:                     7074.
Date:                  Tue, 10 Oct 2023   Prob (F-statistic):               0.00
Time:                          23:53:29   Log-Likelihood:                -40848.
No. Observations:                253661   AIC:                         8.170e+04
Df Residuals:                    253658   BIC:                         8.173e+04
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1544      0.003

# Model #2

# Model #3